In [1]:
from pyspark.sql import SparkSession
from operator import add

In [2]:
# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.127:7077") \
        .appName("2 nodes analysis")\
        .config("spark.executor.cores",1)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-26 14:14:45,347 WARN spark.ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [3]:
from time import perf_counter
from pyspark import SparkFiles

t1_start = perf_counter () 

df = spark_session.read.json('hdfs://192.168.2.127:9000//home/ubuntu/redditdata/RC_2012-01.bz2')

t2 = perf_counter ()

In [4]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from textblob import TextBlob

df_new = df['subreddit','author','body']
sub = ['football', 'soccer', 'English Premier League', 'news']
df_new = df_new.where(df_new.subreddit.isin(sub))
df_new = df_new.where(df_new['body'] != '[deleted]')
df_new = df_new.where(df_new['body'].contains('Manchester City'))
df_sentiment = df_new

def sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    # sentiment = sia.polarity_scores(text)['compound']
    return sentiment

udf_sentiment = udf(sentiment, StringType())

# adding 'sentiment' column
df_sentiment = df_sentiment.withColumn("sentiment", udf_sentiment("body"))
# df_sentiment.agg({'sentiment': 'avg'}).show()
df_sentiment.show()
t3 =  perf_counter ()

+---------+-------------------+--------------------+--------------------+
|subreddit|             author|                body|           sentiment|
+---------+-------------------+--------------------+--------------------+
|   soccer|        ajafarzadeh|As a neutral (wit...| 0.12600000000000003|
|   soccer|         WaiXingRen|Wow as a new fan ...| 0.22624950806768984|
|   soccer|          night_owl|but history dicta...|  0.1136063936063936|
|   soccer|         drwormtmbg|that's because, a...|-0.19999999999999998|
|   soccer|            ravniel|I think this over...| 0.05873015873015873|
|   soccer|               okem|Stats from 2010-1...|                0.36|
|   soccer|             dsemaj|Vincent Kompany h...| 0.27499999999999997|
|   soccer|          [deleted]|I cannot wait to ...|               0.225|
|   soccer|       WineForMyMen|Oh, another Manch...| 0.26249999999999996|
|   soccer|Heinz_Doofenshmirtz|But Joe Hart has ...| 0.26296296296296295|
|   soccer|         Clark-Kent|Manches

In [6]:
print("Total time taken: ", (t3 - t1_start))
print("File loading: ", (t2 - t1_start))

Total time taken:  488.3119136940222
File loading:  346.04334533796646
